In [ ]:
import datetime
import gc
import glob
import os
import random
import re
import tarfile

import flash
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
import sklearn
import torch
import torchvision
from ipynb.fs.defs.dcgan import fix_seed, init_weight
from PIL import Image
from sklearn.datasets import fetch_openml
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [ ]:
SEED = 2913
fix_seed(SEED)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

# Self-Attention GAN

Self-Attention GAN(SAGAN)は、`Self-Attention`, `Pointwise Convolution`, `Spectral Normalization`の3つの技術を軸に構成される。これらの内容は難しいため、書籍からさらに説明を深めて具体例を用いながら説明してく。

## Self-Attention

GANの`ConvTranspose2d`の課題は、入力データの局所的な情報しか使われないことであった。

例えば以下のように`kernel_size=2`の場合には、入力の各ピクセルは、出力の2x2のピクセルにしか影響を与えない。逆に、出力の各ピクセルはせいぜい2つの入力ピクセルからしか影響を受けない。

したがって、入力の全体情報が使われていない。この問題に対処するためにSelf-Attentionが応用される。

In [ ]:
x = torch.rand(1, 2, 2)
print("--------x--------")
display(x)
t = torch.nn.ConvTranspose2d(1, 1, kernel_size=2)
print("--------weight--------")
display(t.weight)
print("--------bias--------")
display(t.bias)
print("--------output--------")
display(t(x))

Self-Attentionでは、あるレイヤーに入力する前に、画像を以下のように変換する。

$
y = x + \gamma o
$

xは入力画像、$\gamma$は係数(学習時に決定される)、$o$はSelf-Attention Mapである。

Self-Attention Mapには、「大域的な」情報が含まれている。ここが少しわかりづらいので、以下、具体的な計算例を見ていきながら、Self-Attention Mapについて理解することにする。

*Todo:xとoを足すと色々と情報が失われる気がする。なぜ別々に入力しないのか？*

In [ ]:
def show_image(img, gray=False, scale=2):
    # set figsize
    plt.figure(figsize=(scale * img.shape[-1], scale * img.shape[-2]))
    channel = img.shape[1]
    if gray:
        plt.gray()
        plt.imshow(img.squeeze(0))
    else:
        plt.imshow(img.squeeze(0).permute(1, 2, 0))

    if not gray:
        for i in range(channel):
            for j in range(img.shape[2]):
                for k in range(img.shape[3]):
                    color = (
                        "black" if torch.mean(img[0, :, j, k]).item() > 0.5 else "white"
                    )
                    plt.text(
                        k,
                        j + 0.2 * (i - 1),
                        "{:.2f}".format(img[0, i, j, k].item()),
                        ha="center",
                        va="center",
                        fontsize=8 * scale,
                        color=color,
                    )
    if gray:
        for j in range(img.shape[1]):
            for k in range(img.shape[2]):
                color = (
                    "black"
                    if (img[0, j, k].item() - torch.min(img).item())
                    / (torch.max(img).item() - torch.min(img).item())
                    > 0.5
                    else "white"
                )
                plt.text(
                    k,
                    j,
                    "{:.2f}".format(img[0, j, k].item()),
                    ha="center",
                    va="center",
                    fontsize=8 * scale,
                    color=color,
                )
    plt.show()

以下の例では、入力xは(1,3,4,4)のサイズのテンソルであるとする。dim=0はバッチサイズ、dim=1はRGBのチャネルの次元、dim=2,3は画像の高さ・幅の次元となっている。

In [ ]:
img_size = 4
# C x H x W
x = torch.rand(1, 3, img_size, img_size)
display(x.shape)
display(x)
show_image(x)

まず、チャネルの次元を残しつつ、高さ・幅の次元を1つにすることで、サイズが(1,3,16)のテンソルx_flattenを作成する。

`x_flatten[0,:,i]`は、各indexのRGBのチャネル情報となる。

ここで、以降の説明をわかりやすくするため、`x_flatten`でフラット化された各ピクセルのチャネル情報を以下のように表すとする。

$
x\_flatten_i = \begin{pmatrix} r_i \\ g_i \\ b_i \end{pmatrix}
$

In [ ]:
x_flatten = x.reshape(1, x.shape[1], -1)
print("--------x_flatten--------")
display(x_flatten.shape)
display(x_flatten)
show_image(x_flatten.unsqueeze(2))
show_image(x_flatten, gray=True)

`x_flatten`の転置行列`x_flatten_t`を作成する。

In [ ]:
x_flatten_t = x_flatten.transpose(1, 2)
print("--------x_flatten_t--------")
display(x_flatten_t.shape)
display(x_flatten_t)
show_image(x_flatten_t, gray=True, scale=0.7)

`x_flatten_t`と`x_flatten`の積をとって、`s`を作成する。この`s`は[グラム行列](https://ja.wikipedia.org/wiki/%E3%82%B0%E3%83%A9%E3%83%A0%E8%A1%8C%E5%88%97)と呼ばれる。グラム行列は、以下のような性質を持つ。

- 正方行列(ここでのサイズは`(16,16)`)
- 正則行列
- 対称行列
- $s_{i,j}$は、$x\_{flatten}_i \cdot x\_{flatten}_j$となる。すなわち、`s`の各成分は、元の画像のピクセルのチャネル情報ベクトル同士に対して内積をとったものになっている。
- なお、チャネル情報の内積を取っているのだから、内積の計算に使用した2つのチャネルベクトルの大きさで内積値を割ることにより、$\cos{\theta_{i,j}} = x\_{flatten}_i \cdot x\_{flatten}_j / (|x\_{flatten}_i|\cdot|x\_{flatten}_j|)$となるから、2つのチャネル情報ベクトルの成す角が計算される。これは相関係数を計算していることに他ならない。
- 上記より、`s`には各チャネル情報の類似度のような情報が含まれていることがわかる。

In [ ]:
s = torch.bmm(x_flatten_t, x_flatten)
display(s.shape)
display(s)
show_image(s, gray=True)

本当に`s`の各成分がチャネル情報の内積になっているのかを確認する。

In [ ]:
eps = 1e-5
for i in range(img_size * img_size):
    for j in range(img_size * img_size):
        diff = abs(
            torch.dot(x_flatten[0, :, i], x_flatten[0, :, j]).item() - s[0, i, j].item()
        )
        assert diff < eps

行方向に対してSoftmax関数による正規化を行う。これは、Attention Mapを転置したものになる(まだ`o`(**Self-**Attention Mapではないので注意)。

*Todo:この正規化は必須？*

In [ ]:
m = torch.nn.Softmax(dim=-2)
attention_map_t = m(s)
print("--------attention_map_t--------")
display(attention_map_t.shape)
# display(attention_map_t)
# show_image(attention_map_t, gray=True)

In [ ]:
attention_map = attention_map_t.transpose(1, 2)
print("--------attention_map--------")
display(attention_map.shape)
display(attention_map)
show_image(attention_map, gray=True)

最後に、x_flattenとAttention Mapの積を取る。これにより、各ピクセルは、そのピクセルと同じような色が多ければ大きくなる。

In [ ]:
o = torch.bmm(x_flatten, attention_map_t)
# make the size of o same as x
o = o.reshape(1, x.shape[1], x.shape[2], x.shape[3])
print("--------o--------")
display(o.shape)
display(o)
show_image(o)

ここまでの処理をクラスとして実装する。書籍とは変えて、SelfAttentionクラスにはSelfAttentionの実装のみを書く(Pointwise Convは別で書く)。

In [ ]:
class PrimitiveSelfAttention(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.softmax = torch.nn.Softmax(dim=-2)
        self.gamma = torch.nn.Parameter(torch.zeros(1))

    def forward(self, x):
        x_flatten = x.view(x.shape[0], x.shape[1], -1)
        x_flatten_t = x_flatten.permute(0, 2, 1)
        s = torch.bmm(x_flatten_t, x_flatten)
        attention_map_t = self.softmax(s)
        attention_map = attention_map_t.permute(0, 2, 1)
        o = torch.bmm(x_flatten, attention_map_t).view(
            x.shape[0], x.shape[1], x.shape[2], x.shape[3]
        )
        return x + self.gamma * o, attention_map

Lenaの画像に対して、Attention Mapを確認してみる。

Lenaの画像テンソルのサイズは、(3,64,64)にしている。したがって、Attention Mapのサイズは、$(64*64, 64*64)=(4096,4096)$になる。Attention Mapのサイズはかなり大きくなり、入力画像のサイズが大きすぎる時にはメモリに乗らなくなるので注意。

下記の実装で示している通り、`attention_map[0,:,x*img_size+y]`の1次元ベクトルを2次元ベクトルに変換すると、(x,y)に対応したAttentionを見るおがことができる。下図の可視化では、ランダムに選ばれた"X"のマーカーに対するAttentionをヒートマップで表示している。オリジナルの画像と見比べると、マーカーの位置と同じような色の部分が濃く(より赤色に)表示されていることがわかる。

In [ ]:
# download lena from https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png
if not os.path.exists("data/lena.png"):
    url = "https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png"
    response = requests.get(url)
    with open("data/lena.png", "wb") as file:
        file.write(response.content)

img_size = 64
# open lena and convert it to tensor
lena = Image.open("data/lena.png")
lena = torchvision.transforms.Resize((img_size, img_size))(lena)
lena = torchvision.transforms.ToTensor()(lena).to(device)
self_attention = PrimitiveSelfAttention().to(device)
_, attention_map = self_attention(lena.unsqueeze(0))

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(lena.detach().cpu().permute(1, 2, 0))
ax.set_title("original")
plt.show()

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
fig.suptitle("attention map")
for ax in axes.flatten():
    # random sample from0~img_size
    x = random.randint(0, img_size - 1)
    y = random.randint(0, img_size - 1)
    # ax.imshow(attention_map[0, :, x*img_size+y].detach().cpu().reshape(img_size, img_size))
    sns.heatmap(
        attention_map[0, :, x * img_size + y]
        .detach()
        .cpu()
        .reshape(img_size, img_size),
        ax=ax,
        cbar=False,
        cmap="jet",
    )
    # highlight x,y
    ax.scatter(y, x, c="black", s=100, marker="x")
plt.tight_layout()
plt.show()

In [ ]:
del lena, attention_map, self_attention
gc.collect()
torch.cuda.empty_cache()

## Pointwise Convolution

Pointwise Convolutionはkernel_sizeが1のあた畳み込み。画像の場合であれば、高さ・幅の次元は変化させないままに、チャネルの次元を変えることができる。kernel_size=1なので、周辺情報を含めた特徴量の作成はできないものの、チャネルの出力次元を入力次元を小さくすることにより、次元の圧縮を行うことができる。

先ほどのLenaの画像を用いたSelf Attentionでも説明した通り、Attention Mapのサイズはかなり大きくなる。そこで、Pointwise Convolutionによる次元圧縮を行うことにより、Attention Mapのサイズを小さくしつつ、Self-Attention Layerを適用できることがポイントになる。

Self-Attentionでは、`key`, `query`, `value`という用語が一般的らしい。基本的な操作は上記と同じであるものの、`key`, `query`, `value`のそれぞれに対してPointwise Convolutionを行うことがポイント。詳細は下図。

```mermaid
flowchart LR;

input --[Pointwise Conv]--> key ----> dot1([x])
input --[Pointwise Conv]--> query --[transpose]-->query_t ----> dot1 --[Softmax]--> attention_map_t ----> dot2([x]) 
input --[Pointwise Conv]--> value ----> dot2 ----> o

```

In [ ]:
class SelfAttention(torch.nn.Module):
    def __init__(self, in_channels, feat_channels):
        super().__init__()
        self.softmax = torch.nn.Softmax(dim=-2)
        self.gamma = torch.nn.Parameter(torch.zeros(1))
        self.convs = torch.nn.ModuleDict(
            {
                key: torch.nn.Conv2d(
                    in_channels=in_channels,
                    out_channels=feat_channels if key != "value" else in_channels,
                    kernel_size=1,
                )
                for key in ["key", "query", "value"]
            }
        )

    def forward(self, x):
        batch_size, _, height, width = x.shape
        key = self.convs["key"](x).view(batch_size, -1, height * width)
        query = (
            self.convs["query"](x).view(batch_size, -1, height * width).permute(0, 2, 1)
        )
        s = torch.bmm(query, key)

        attention_map_t = self.softmax(s)
        attention_map = attention_map_t.permute(0, 2, 1)

        value = self.convs["value"](x).view(batch_size, -1, height * width)
        o = torch.bmm(value, attention_map_t).view(batch_size, -1, height, width)

        return x + self.gamma * o, o, attention_map

In [ ]:
_batch_size = 4
_input = torch.rand(_batch_size, 3, 64, 64)
_self_attention = SelfAttention(3, 1)
_output, _o, _attention_map = _self_attention(_input)
assert _output.shape == _input.shape
assert _o.shape == _input.shape
assert _attention_map.shape == (_batch_size, 64 * 64, 64 * 64)
del _input, _output, _o, _attention_map, _self_attention
gc.collect()
torch.cuda.empty_cache()

## Spectral Normalization

`wgan.ipynb`では、Discriminatorのリプシッツ連続性が重要であると述べた。まず、リプシッツ連続性について理解する。

ある関数$f(x)$がリプシッツ連続であるとは、任意の$x_1, x_2$に対して、$|f(x_1) - f(x_2)| \leq K |x_1 - x_2|$が成立するリプシッツ定数$K$が存在することをいう。これを定性的にとらえると、入力の変化に対して出力の変化は上限があるということになる。

具体例で考えると、$f(x)=x^2$はリプシッツ連続ではないが、$f(x)=\sqrt(x)$はリプシッツ連続である。$x^2$は$x$が大きくなるにつれて急激に変化するものの、$\sqrt(x)$はそうではないので直感的に正しそうなことがわかると思う。なお、証明も難しくない。

Discriminator, Generatorがリプシッツ連続であれば、入力が多少変化したとしても出力が大きく変化することはない。そこで、これらをリプシッツ連続にするために、Spectral NormalizationをGenerator, Discriminatorの重みに適用する。これにより、リプシッツ定数を1以下に制限することができる。

Pytorchでは、`torch.nn.utils.spectral_norm`を使用することで実装が簡単になる。

以下の実装では、WGANと同様にWasserstein Lossを使用するので、出力は書籍と異なる。

In [ ]:
class Deconv2d(torch.nn.Sequential):
    def __init__(self, in_channels, out_channels, with_batch_norm=True, **kwargs):
        super().__init__()
        self.add_module(
            "ConvTranspose2d",
            torch.nn.utils.spectral_norm(
                torch.nn.ConvTranspose2d(in_channels, out_channels, **kwargs)
            ),
        )
        if with_batch_norm:
            self.add_module("BatchNorm2d", torch.nn.BatchNorm2d(out_channels))
        self.add_module("ReLU", torch.nn.ReLU(inplace=True))

In [ ]:
class Generator(torch.nn.Sequential):
    def __init__(self, z_dim, image_size):
        super().__init__()

        self.layers = torch.nn.ModuleList(
            [
                Deconv2d(z_dim, image_size * 8, kernel_size=4, stride=1),
                Deconv2d(
                    image_size * 8, image_size * 4, kernel_size=4, stride=2, padding=1
                ),
                Deconv2d(
                    image_size * 4, image_size * 2, kernel_size=4, stride=2, padding=1
                ),
                SelfAttention(image_size * 2, image_size * 2 // 8),
                Deconv2d(
                    image_size * 2, image_size, kernel_size=4, stride=2, padding=1
                ),
                SelfAttention(image_size, image_size // 8),
                Deconv2d(
                    image_size,
                    1,
                    kernel_size=4,
                    stride=2,
                    padding=1,
                    with_batch_norm=False,
                ),
            ]
        )

    def forward(self, x):
        supplement = {}
        for i, layer in enumerate(self.layers):
            if isinstance(layer, SelfAttention):
                x, o, attention_map = layer(x)
                supplement[f"o_{i+1}"] = o.detach().cpu()
                supplement[f"attention_map_{i+1}"] = attention_map.detach().cpu()
            else:
                x = layer(x)
        if self.training:
            return x
        return x, supplement

In [ ]:
_batch_size = 4
_input = torch.rand(_batch_size, 20, 1, 1)
_generator = Generator(20, 64)
_output = _generator(_input)
assert _output.shape == (_batch_size, 1, 64, 64)
gc.collect()
torch.cuda.empty_cache()

In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self, image_size):
        super().__init__()
        self.layers = torch.nn.ModuleList(
            [
                torch.nn.utils.spectral_norm(
                    torch.nn.Conv2d(1, image_size, kernel_size=4, stride=2, padding=1)
                ),
                torch.nn.LeakyReLU(0.1, inplace=True),
                torch.nn.utils.spectral_norm(
                    torch.nn.Conv2d(
                        image_size, image_size * 2, kernel_size=4, stride=2, padding=1
                    )
                ),
                torch.nn.LeakyReLU(0.1, inplace=True),
                torch.nn.utils.spectral_norm(
                    torch.nn.Conv2d(
                        image_size * 2,
                        image_size * 4,
                        kernel_size=4,
                        stride=2,
                        padding=1,
                    )
                ),
                torch.nn.LeakyReLU(0.1, inplace=True),
                SelfAttention(image_size * 4, image_size * 4 // 8),
                torch.nn.utils.spectral_norm(
                    torch.nn.Conv2d(
                        image_size * 4,
                        image_size * 8,
                        kernel_size=4,
                        stride=2,
                        padding=1,
                    )
                ),
                torch.nn.LeakyReLU(0.1, inplace=True),
                SelfAttention(image_size * 8, image_size * 8 // 8),
                torch.nn.utils.spectral_norm(
                    torch.nn.Conv2d(
                        image_size * 8, 1, kernel_size=4, stride=1, padding=0
                    )
                ),
            ]
        )

    def forward(self, x):
        supplement = {}
        for i, layer in enumerate(self.layers):
            if isinstance(layer, SelfAttention):
                x, o, attention_map = layer(x)
                supplement[f"o_{i+1}"] = o.detach().cpu()
                supplement[f"attention_map_{i+1}"] = attention_map.detach().cpu()
            else:
                x = layer(x)
        if self.training:
            return x.view(-1)
        return x.view(-1), supplement

In [ ]:
_batch_size = 4
_input = torch.rand(_batch_size, 1, 64, 64)
_discriminator = Discriminator(64)
_output = _discriminator(_input)
assert _output.shape == (_batch_size,)
del _input, _output, _discriminator
gc.collect()
torch.cuda.empty_cache()

## 学習

Self-Attention層でメモリの使用量が多くなるため、バッチサイズをあまり大きくできないことに注意。

In [ ]:
class MnistTransform:
    interpolation_modes = [
        torchvision.transforms.InterpolationMode.NEAREST,
        torchvision.transforms.InterpolationMode.NEAREST_EXACT,
        torchvision.transforms.InterpolationMode.BILINEAR,
        torchvision.transforms.InterpolationMode.BICUBIC,
    ]

    def __init__(self, image_size):
        self.transform = torchvision.transforms.Compose(
            [
                torchvision.transforms.RandomChoice(
                    [
                        torchvision.transforms.Resize(
                            (image_size, image_size), interpolation=mode
                        )
                        for mode in self.interpolation_modes
                    ]
                ),
                torchvision.transforms.RandomRotation(10),
                torchvision.transforms.ToTensor(),
                # add random noize, std=0.1
                torchvision.transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.01),
            ]
        )

    def __call__(self, x):
        return self.transform(x)

In [ ]:
image_size = 64
transform = MnistTransform(image_size)
train_dataset = torchvision.datasets.MNIST(
    root="./data", train=True, transform=transform, target_transform=None, download=True
)
val_dataset = torchvision.datasets.MNIST(
    root="./data",
    train=False,
    transform=transform,
    target_transform=None,
    download=True,
)
dataset = train_dataset + val_dataset
batch_size = 512
train_data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=os.cpu_count(),
    pin_memory=True,
    persistent_workers=True,
)

In [ ]:
z_dim = 20
num_epochs = 100
n_critics = 1

num_image_to_save = 8 * 4
z_fixed = torch.randn(num_image_to_save, z_dim, 1, 1, device=device)

generator = Generator(z_dim, image_size).to(device)
discriminator = Discriminator(image_size).to(device)
generator.apply(init_weight)
discriminator.apply(init_weight)
scaler = torch.cuda.amp.GradScaler()


start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

lr_d, betas_d = 0.0001, (0.5, 0.999)
lr_g, betas_g = 0.0001, (0.5, 0.999)
optimizer_d = flash.core.optimizers.LAMB(
    discriminator.parameters(), lr=lr_d, betas=betas_d
)
optimizer_g = flash.core.optimizers.LAMB(generator.parameters(), lr=lr_g, betas=betas_g)


torch.backends.cudnn.benchmark = True

datetime_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
save_dir = f"./sagan/{datetime_str}"
os.makedirs(save_dir, exist_ok=True)

writer = SummaryWriter(log_dir=os.path.join(save_dir, "logs"))
writer.add_text("Config/discriminator", f"lr_d: {lr_d}, betas_d: {betas_d}")
writer.add_text("Config/generator", f"lr_g: {lr_g}, betas_g: {betas_g}")
writer.add_text(
    "Config/Common",
    f"z_dim: {z_dim}, n_critics: {n_critics}, batch_size: {batch_size}, num_epochs: {num_epochs}",
)

In [ ]:
generator = generator.to(device)
discriminator = discriminator.to(device)

print("Start training")
for epoch in range(num_epochs):
    start.record()
    minibatch_d_losses = []
    minibatch_g_losses = []

    generator.train()
    discriminator.train()

    for i, (images, _) in enumerate(train_data_loader):
        mini_batch = images.size()[0]
        images = images.to(device)
        if mini_batch == 1:
            continue

        for _ in range(n_critics):
            # train discriminator
            discriminator.zero_grad()
            with torch.cuda.amp.autocast(dtype=torch.float16):
                z = torch.randn(mini_batch, z_dim, 1, 1, device=device)

                fake_images = generator(z)
                loss_d_real = discriminator(images).mean()
                loss_d_fake = discriminator(fake_images.detach()).mean()
                loss_d = -loss_d_real + loss_d_fake
            scaler.scale(loss_d).backward()
            scaler.step(optimizer_d)
            scaler.update()
            minibatch_d_losses.append(loss_d.item())
            for p in discriminator.parameters():
                p.data.clamp_(-0.01, 0.01)

        # train generator
        generator.zero_grad()
        with torch.cuda.amp.autocast(dtype=torch.float16):
            loss_g = -discriminator(fake_images).mean()
        scaler.scale(loss_g).backward()
        scaler.step(optimizer_g)
        scaler.update()
        minibatch_g_losses.append(loss_g.item())

    end.record()
    torch.cuda.synchronize()
    print(
        f"Epoch: {epoch+1}/{num_epochs}, d_loss: {np.mean(minibatch_d_losses):.4f}, g_loss: {np.mean(minibatch_g_losses):.4f}, time: {start.elapsed_time(end)/1000:.4f} sec"
    )

    writer.add_scalars(
        "Loss",
        {
            "discriminator": np.mean(minibatch_d_losses),
            "generator": np.mean(minibatch_g_losses),
        },
        epoch,
    )
    writer.add_scalar("Time", start.elapsed_time(end) / 1000, epoch)
    generator.eval()
    fake_images, supplement = generator(z_fixed)
    writer.add_images(
        "Images/generator",
        fake_images.detach().cpu(),
        epoch,
    )
    for key in ["attention_map_4", "attention_map_6"]:
        am_size = int(np.prod(supplement[key].shape[1:], axis=0) ** 0.25)
        ams = supplement[key][0].squeeze().view([am_size for _ in range(4)])
        # choose 8 from 0~am_size uniformly
        xs = np.linspace(0, am_size - 1, 8).astype(int)
        ys = np.linspace(0, am_size - 1, 8).astype(int)
        writer.add_image(
            f"Images/generator_{key}",
            ams[xs, ys, :, :].unsqueeze(1),
            epoch,
            dataformats="NCHW",
        )

## 学習結果の確認

## 参考文献

- [つくりながら学ぶ！PyTorchによる発展ディープラーニング | 小川 雄太郎 | 工学 | Kindleストア | Amazon](https://www.amazon.co.jp/%E3%81%A4%E3%81%8F%E3%82%8A%E3%81%AA%E3%81%8C%E3%82%89%E5%AD%A6%E3%81%B6%EF%BC%81PyTorch%E3%81%AB%E3%82%88%E3%82%8B%E7%99%BA%E5%B1%95%E3%83%87%E3%82%A3%E3%83%BC%E3%83%97%E3%83%A9%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0-%E5%B0%8F%E5%B7%9D-%E9%9B%84%E5%A4%AA%E9%83%8E-ebook/dp/B07VPDVNKW/ref=sr_1_1?__mk_ja_JP=%E3%82%AB%E3%82%BF%E3%82%AB%E3%83%8A&crid=39VBRPTDUUH0F&keywords=%E4%BD%9C%E3%82%8A%E3%81%AA%E3%81%8C%E3%82%89%E5%AD%A6%E3%81%B6+pytorch&qid=1701503265&sprefix=%E4%BD%9C%E3%82%8A%E3%81%AA%E3%81%8C%E3%82%89%E5%AD%A6%E3%81%B6+pytorch%2Caps%2C221&sr=8-1)
- [[1802.05957] Spectral Normalization for Generative Adversarial Networks](https://arxiv.org/abs/1802.05957)